<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/tabular_neural_net_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Model For Waste Data Classification


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.
df - data_frame

In [0]:
# mount to google drive and load files
from google.colab import drive
drive.mount('/content/drive')


In [0]:
# check file is loaded OK
path = '/content/drive/My Drive/data/'
print(path)

In [0]:
# load train data frame
# todo: 1. iterate through nl data and export only with material, eural code fields
#  then translate to en. this will increase train and test data sets and include missing material categories that appear in validation set
df = pd.read_csv(path + '/minimizedData.csv')



# load test data frame with material field empty
test_df = pd.read_csv(path + '/testNoMaterialData.csv')

df.head()

,reason,origin,color,state,size,consistency,otherCode,material4,material3,material2,material,mType,composite2,composite1,cType,indirectProduct,directProduct,pType,mixedOrPure,cleanOrDirty,euralDescription,euralCode,description,/0
0,NaN,NaN,NaN,NaN,NaN,slurry,NaN,NaN,NaN,oil,organic material,Soy,NaN,dry,NaN,dry,NaN,NaN,1.0,NaN,material unsuitable for consumption or processing,20304,Soyadroes technically,NaN
1,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,wood,branches,NaN,NaN,NaN,wood,NaN,NaN,0.0,NaN,waste from forestry,20107,Branches,NaN
2,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,cocoa,caps,NaN,NaN,NaN,cocoa,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,Cocoa shells,NaN
3,NaN,NaN,NaN,debris,NaN,vast,NaN,NaN,NaN,NaN,sand,NaN,NaN,debris,NaN,debris,NaN,NaN,1.0,1.0,"mixtures of concrete, stones, tiles or ceramic...",170107,Debris with Sand,NaN
4,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,organic material,Soy,NaN,NaN,NaN,organic material,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,soya,NaN


#cats and conts 

**categorical variables - cat_names:**

what you're using to make predictions with.  (description, euralDescription, composite1, mType).
for categorical variables - will use Embeddings 

**continuos variables - cont_names:**

any catgeory that has limited number of possible (integer) choises: true/false ; fields such as cleanOrContaminated or mixedOrPure
continuos varibale names - they are sent to Neural net as pixels 

**uses processor - procs:**

a number of processes that deal with data in some ways: FillMissing, Categorify, Normalize both train and test dtata sets

In [0]:
# dependable variable field:
dep_var = 'material'
 
# can add composite1, mType
cat_names = ['euralDescription', 'description', 'euralCode']
cont_names = []
# cont_names = ['mixedOrPure', 'cleanOrDirty']
# processors:
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[950:1200].copy(), path=path + '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names)

Next:
data block api
add labels - tell which cols - column it is 

and get back data bunch
split train and test data ranges 800,1000


In [0]:
data = (TabularList.from_df(df, path=path+ '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(950,1200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
PCFIC, non-iron, solvents, syrup, flint...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [0]:
data.show_batch(rows=10)

euralDescription,description,euralCode,target
paper and cardboard,Envelopes,200101,paper
wood,"B-Wood, painted, glued sheet material",170201,wood
metal,Demolition,200140,metal
building material containing asbestos,"asbestos, not route",170605,asbestos
#na#,2330 b wood,170904,wood
metal waste,Iron,20110,iron
biodegradable waste,Gardening system heavy / chips,200201,organic material
soil and stones not covered by 17 05 03,category 1 soil,170504,ground
plastics,PA / PE - foil,200139,PA
biodegradable waste,Organic waste (NTA 610),200201,organic material


pass in data to tabular learner:


In [0]:
learn = tabular_learner(data, layers=[300,100], metrics=accuracy)

In [0]:
learn.fit(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.900922,2.334291,0.565574,00:00
1,1.188990,1.174783,0.717213,00:00


In [0]:
# see into the model
# learn.model.eval()
learn.summary()


TabularModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [15]                 855        True      
______________________________________________________________________
Embedding            [104]                179,088    True      
______________________________________________________________________
Embedding            [18]                 1,404      True      
______________________________________________________________________
Dropout              [137]                0          False     
______________________________________________________________________
Linear               [300]                41,400     True      
______________________________________________________________________
ReLU                 [300]                0          False     
______________________________________________________________________
BatchNorm1d          [300]                600        True      
_________________________________________________

In [0]:
learn.model_dir = '/content/drive/My Drive/data/models'
learn.save('material_classifier_model')

## Inference  
- run test data set to predict material
- extract the prediction confindece score  

In [0]:
row = 111

# data_frame = df
data_frame = test_df

print('given material -->', data_frame.iloc[row].material)
print('description ---> ', data_frame.iloc[row].description, ', ', data_frame.iloc[row].euralDescription)


given material --> nan
description --->  Notifiable demolition waste ,  mixed construction and demolition wastes other than those mentioned in 17 09 01, 17 09 02 and 17 09 03


In [0]:
#one_row = df.iloc[row]
one_row = test_df.iloc[row]

predicted_cat, total_cat, weights = learn.predict(one_row)
print('predicted material ----> ', predicted_cat)

predicted material ---->  sand


Category Probability - Prediction Confidence score

In [0]:
cat, idx, confidence = learn.predict(one_row)
cat_prob = confidence[idx.item()].item()
print(' category probability --------->', cat_prob)

 category probability ---------> 0.21948103606700897
